# PROJECT 2: CHATBOT (GPT + LLAMA)

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [17]:
# prompts
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: "

# initialize model
def stream_response(question, model, openai):
    stream = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
# Strategy pattern
from abc import ABC, abstractmethod
# abc = abstract base class
# Step 1: Define the Strategy Interface
class OpenAIStrategy(ABC):
    @abstractmethod
    def getStreamResponse(self, question: str) -> None:
        pass

# Step 2: Create Concrete Strategies
class GPTStrategy(OpenAIStrategy):
    def getStreamResponse(self, question: str) -> None:
        openai = OpenAI()
        stream_response(question, MODEL_GPT, openai)

class LLAMAStrategy(OpenAIStrategy):
    def getStreamResponse(self, question: str) -> None:
        openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        stream_response(question, MODEL_LLAMA, openai)

# Step 3: Create the Context
class ApiProcessor:
    def __init__(self, strategy: OpenAIStrategy):
        self._strategy = strategy

    def set_strategy(self, strategy: OpenAIStrategy):
        """Change the strategy at runtime."""
        self._strategy = strategy

    def process_model(self, question: str):
        """Apply the current strategy to the text."""
        self._strategy.getStreamResponse(user_prompt + question)

In [ ]:
# here is the question; type over this to ask something new

question = """
    if age >= 18:
        print("You are an adult.")
    else:
        print("You are a minor.")
"""

In [18]:
# Get gpt-4o-mini to answer, with streaming
apiProcessor = ApiProcessor(GPTStrategy())
apiProcessor.process_model(question)

Sure! Let's break down the code snippet you provided: 

python
yield from {book.get("author") for book in books if book.get("author")}


### Components of the Code

#### 1. Set Comprehension
The code inside the `{}` is a **set comprehension**. Set comprehensions allow you to create a set in a concise way and can be thought of as similar to list comprehensions.

- **`{book.get("author") for book in books if book.get("author")}`**:
  - `book` iterates over each element in `books`.
  - `book.get("author")` tries to retrieve the value associated with the key `"author"` from each `book` dictionary.
  - The `if book.get("author")` clause serves as a filter, ensuring that only books with a non-`None` and non-empty author string are included in the resulting set.

In summary, this expression creates a set containing unique author names from the `books` collection, but only if the author key has a valid (non-empty) value.

#### 2. `yield from`
The `yield from` statement is used to yield all values from an iterable (in this case, the set created by the comprehension) one at a time. This is typically seen in generator functions in Python. 

- This means the function containing this code is likely a generator function, which can produce a series of values over time rather than returning all of them at once. By using `yield`, the function can be paused and resumed, which is useful for handling large datasets or streams of data efficiently.

### Why Use This Code?

1. **Extract Unique Authors**: The set comprehension combined with `get()` and conditional filtering ensures we attain a unique set of authors from the `books` collection while safely handling cases where the `"author"` key might not exist or might have a `None` value.
  
2. **Memory Efficiency**: Using a generator (via `yield from`) allows the code to process and return authors on-the-fly, rather than storing them all in memory at once. This can be particularly important when dealing with large datasets, as it reduces memory usage.

3. **Cleaner Code**: The use of set comprehensions and `yield from` simplifies the syntax compared to manually constructing a set and then separately yielding its contents.

### Example Scenario

Let's say you have a list of books (as dictionaries) like this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
    {"title": "Book 5", "author": ""},
]


If you execute the line of code provided, it would produce a set containing `"Author A"` and `"Author B"` (note that "Author A" appears twice in the input but only once in the output due to the properties of a set).

### Conclusion

Overall, this single line of code is effective for extracting a set of unique author names from a collection of book references while using a memory-efficient approach provided by Python’s generator functionality. It elegantly handles missing data and avoids duplication.

In [19]:
# Get Llama 3.2 to answer
apiProcessor = ApiProcessor(LLAMAStrategy())
apiProcessor.process_model(question)

**Code Analysis**

The given code snippet involves a combination of Python's yield from syntax, dictionary lookup (`get()`), and list comprehension. Let's break it down:

python
yield from {...}


In Python 3.3 and later versions, `yield from` is used to delegate the iteration of an iterable (either a single object or a generator) to another iterable. It creates a new generator that iterates over the given iterable, rather than yielding its own values.

python
...
for book in books if condition:
    ...


Here, the code uses a conditional statement (`if`) to filter books based on an author-related check. If the resulting iterator is assigned to `books`, it implies that we might be working with an extended Iterable-like object that satisfies the conditions of the original iterable.

Now, let's consider:

python
{book.get("author") for book in books if book.get("author")}


This is a dictionary comprehension (although not as traditional as others) using a conditional `if` to filter items in the iterable (`books`). It takes each dictionary item (`book`) inside the iterable, extracts its author value from it using `.get("author")`, and uses these value-only dictionaries for iteration.

Finally:

python
yield from {...}


In this context, `yield from {book.get("author") for book in books if book.get("author")}` is performing three primary tasks: 

    1. Filtering: the conditional part `if book.get("author")` iterates only over those dictionaries (`books`) that have a value present in `"author"`-key;

2. Dictionary values extraction and processing of key-value pairing into a value-only dictionary by `.get("author")`, i.e., `{book author:`}, effectively discarding every other value or pairs.
However for yielding this list of extracted books authors, yield from {....} acts in place of for loop that iterates over it. This lets you process multiple elements like 'author' (i.e., list comprehension and not `for` iteration to do so.



**Code Explanation with Code**

Here is the equivalent code explanation:
python
def get_authors_from_books(books):
    for book in books:
        if "author" in book:
            yield book["author"]

In this code, if we were using traditional for loop instead of `yield from`: 
But we would still use similar filtering with `.get("author")` when defining this variable. With dictionary comprehension to further strip other value pairs if necessary.

Here's an example usage:

python
# Sample books data as dictionaries
books = [
    {"id": 1, "title": "Book A", "author": "Author 1"},
    {"id": 2, "book": "Another text Book", "author": "Author 3"},
    {"id": 3, "text": "Third book chapter", "author": "Author 4"}
]

# Function to get filtered authors using dictionary literal `get()`
def get_authors(books):
    return {book.get("author") for book in books if book.get("author")}

print(get_authors(books)) 


**Answer** The final output list is: 


{'Author 1', 'Author 3', 'Author 4'}
